In [21]:
from neo4j import GraphDatabase

In [22]:
URI = "neo4j+s://67ec8314.databases.neo4j.io"
AUTH = ("neo4j", "ptdQYuJCwqQQt6BOO-qi323XZjT4G8wEnh90ohOrX3A")


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [23]:
from flask import Flask, request
from py2neo import Graph, NodeMatcher
import networkx as nx
import json

In [24]:
app = Flask(__name__)

# Connect to Neo4j database
graph = Graph(URI, auth=AUTH)


@app.route('/graph-data')
def get_graph_data():
    
    full_name = request.args.get('full_name')

    matcher = NodeMatcher(graph)
    id_node = matcher.match('ID', name=full_name).first()
    name1_node = matcher.match('NAME1', name=full_name).first()
    name2_node = matcher.match('NAME2', name=full_name).first()
    

    # Create NetworkX graph and add nodes and edges
    nx_graph = nx.Graph()
    if id_node is not None:
        nodes = [id_node]
        edges = []
        for rel in graph.match(nodes=[id_node], r_type=None):
            nodes.append(rel.end_node)
            edges.append((id_node, rel.end_node, {'label': rel.type}))
        for node in nodes:
            nx_graph.add_node(node.id, label=node['name'], type='ID')
        for edge in edges:
            nx_graph.add_edge(edge[0].id, edge[1].id)
    if name1_node is not None:
        nodes = [name1_node]
        edges = []
        for rel in graph.match(nodes=[name1_node], r_type=None):
            nodes.append(rel.end_node)
            edges.append((name1_node, rel.end_node, {'label': rel.type}))
        for node in nodes:
            nx_graph.add_node(node.id, label=node['name'], type='NAME1')
        for edge in edges:
            nx_graph.add_edge(edge[0].id, edge[1].id)
    if name2_node is not None:
        nodes = [name2_node]
        edges = []
        for rel in graph.match(nodes=[name2_node], r_type=None):
            nodes.append(rel.end_node)
            edges.append((name2_node, rel.end_node, {'label': rel.type}))
        for node in nodes:
            nx_graph.add_node(node.id, label=node['name'], type='NAME2')
        for edge in edges:
            nx_graph.add_edge(edge[0].id, edge[1].id)

    
    if 'application/graphml+xml' in request.headers.get('Accept'):
        # Converting to GraphML format
        graphml_data = nx.generate_graphml(nx_graph)
        return graphml_data
    else:
        # Converting to JSON format
        nodes = [{'id': n[0], 'label': n[1]['label'], 'type': n[1]['type']} for n in nx_graph.nodes(data=True)]
        edges = [{'source': e[0], 'target': e[1], 'label': e[2]['label']} for e in nx_graph.edges(data=True)]
        json_data = json.dumps({'nodes': nodes, 'edges': edges})
        return json_data, {'Content-Type': 'application/json'}

    
if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-03-17 12:11:15,890] ERROR in app: Exception on /graph-data [GET]
Traceback (most recent call last):
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Irina\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Iri